In [ ]:
%pip install crewai_tools crewai -U

In [ ]:
%pip install --upgrade crewai crewai_tools

In [ ]:
pip show crewai

In [ ]:
%pip install litellm

In [ ]:
# Initialize all your components
from crewai import Crew, Process, LLM
#import agentops
from dotenv import load_dotenv
import os
import json
#from datetime import datetime
# Import your other agents from their respective files
from Investigator_Agent2 import selected_agent
from Requirement_Scout_Agent1 import Extractor_agent
#Key Needed
from openai import OpenAI
load_dotenv()

#LLM

groq_llm = LLM(
    model="groq/llama-3.1-8b-instant",
    temperature=0.1
)
groq_llm2 = LLM(
    model="groq/llama-3.3-70b-versatile",
    temperature=0.1
)

openai_llm = LLM(
    model="openai/gpt-4o-mini", 
    temperature=0.1
)


#main.py
def main():
    #from crewai.knowledge.source.string_knowledge_source import StringKnowledgeSource

    # Initialize the search recommender
    output_dir=r"C:\Users\user\OneDrive - University of Prince Mugrin\سطح المكتب\Capstone_Project_SDAIA\src\outputs"
    Extractor= Extractor_agent(llm=openai_llm, output_dir=output_dir)
    

    #context_dependency
   
    #knowledge_sources
    """about_file = "ohay is a company that provides AI solutions to help websites refine their search and recommendation systems."
    file_context = StringKnowledgeSource(
        content=about_file
    )"""

    # Create the crew
    crew = Crew(
        agents=[
            Extractor.get_agent,
    
        ],
        tasks=[
            Extractor.get_task,
            
        ],
        process=Process.sequential,
        max_rpm=1,
        
        #knowledge_sources=[file_context],  # Make sure company_context is defined
        #memory=True
    )
    
   
    result = crew.kickoff()

    print("=== RESULTS ===")
    print(result)
    return result

if __name__ == "__main__":
    main()

In [ ]:
import os
from openai import OpenAI
from dotenv import load_dotenv

# تحميل المتغيرات من ملف .env
load_dotenv()

# استدعاء المفتاح
api_key = os.getenv("OPENAI_API_KEY")

# تمرير المفتاح للعميل
client = OpenAI(api_key=api_key)

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": "Hello"}]
)

print(response.choices[0].message.content)

In [ ]:
import socket
try:
    host = socket.gethostbyname("api.groq.com")
    print(f"Connection successful: {host}")
except Exception as e:
    print(f"Connection failed: {e}")

In [ ]:
import re

def get_sections_content(md_file_path, selected_headings):
    with open(md_file_path, 'r', encoding='utf-8') as f:
        content = f.read()

    extracted_data = {}
    
    # تقسيم الملف بناءً على العناوين (Headers) التي تبدأ بـ # أو ## أو ###
    # سنبحث عن العناوين المختارة ونستخرج ما تحتها
    for heading in selected_headings:
        # نمط Regex للبحث عن العنوان وما بعده حتى العنوان التالي
        # نستخدم re.escape للتعامل مع أي رموز خاصة في اسم العنوان
        escaped_heading = re.escape(heading)
        pattern = rf"(^#+.*{escaped_heading}.*$\n)([\s\S]*?)(?=\n#+|$)"
        
        match = re.search(pattern, content, re.MULTILINE | re.IGNORECASE)
        if match:
            extracted_data[heading] = match.group(2).strip()
        else:
            extracted_data[heading] = "لم يتم العثور على محتوى لهذا القسم."

    return extracted_data

In [ ]:
selected_titles = [" الاستبعاد من المنافسة"," الموافقة على الشروط"]
filtered_context = get_sections_content(r"C:\Users\user\OneDrive - University of Prince Mugrin\سطح المكتب\Capstone_Project_SDAIA\src\Agents\request1_final_clean3.md", selected_titles)

In [ ]:
filtered_context

In [ ]:
load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")

if not api_key:
    raise ValueError("لم يتم العثور على GOOGLE_API_KEY. تأكد من إضافته لملف .env")


In [ ]:
%pip install "unstructured[md]"

In [ ]:
import os
from agentic_doc.parse import parse

# 1. تحديد المسارات
pdf_path = r"C:\Users\user\OneDrive - University of Prince Mugrin\سطح المكتب\Capstone_Project_SDAIA\src\data\request1.pdf"
output_folder = r"C:\Users\user\OneDrive - University of Prince Mugrin\سطح المكتب\Capstone_Project_SDAIA\src\data\processed"

# إنشاء المجلد إذا لم يكن موجوداً
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# 2. عملية التحويل
print("جاري تحويل الملف من PDF إلى MD...")
results = parse([pdf_path])

# 3. حفظ النتيجة كملف .md
# نفترض أن results عبارة عن قائمة كائنات، نأخذ النص من العنصر الأول
md_filename = "request1_raw.md"
md_output_path = os.path.join(output_folder, md_filename)

with open(md_output_path, "w", encoding="utf-8") as f:
    # ملاحظة: تأكد من مخرجات مكتبة agentic_doc، عادة النص يكون في .text أو .markdown
    f.write(results[0].markdown) 

print(f"✅ تم حفظ الملف الخام في: {md_output_path}")

clean 

In [ ]:
import re
from unstructured.partition.md import partition_md
from unstructured.cleaners.core import clean_extra_whitespace

def auto_clean_with_unstructured(file_path):
    # 1. تقسيم الملف إلى عناصر
    elements = partition_md(filename=file_path)

    cleaned_text = []

    for element in elements:
        text = element.text.strip()
        
        # --- الفلترة بناءً على بداية النص (أوصاف الصور واللوقو) ---
        image_keywords = [
            "Summary :", "logo:", "Visible Elements :", "Placement & Dimensions :", 
            "Analysis :", "Graphic Elements :", "Text Elements :", "Design & Placement :", 
            "Design & Layout :", "Layout :", "Design Elements :", "Design Details :", 
            "Text Fields :", "Colour Palette :", "Spatial Relationships :","Dimensions & Placement :","/25, 4:29 PM","تاريخ طرح الكراسة: /1446","/1446"
        ]
        
        if any(text.startswith(key) for key in image_keywords):
            continue 

        # --- الفلترة بناءً على وجود كلمات معينة (الترويسات والتذييلات) ---
        excluded_headers = [
            "المملكة العربية السعودية", "كراسة الشروط", "تاريخ الإصدار", 
            "tenders.etimad.sa", "المعتمد بموجب قرار وزير المالية","اسم المنافسة: ","رقم الكراسة: ","تاريخ طرح الكراسة:" ,"/25, 4:29 PM"
        ]
        if any(key in text for key in excluded_headers):
            continue

        # --- تصحيح حذف أرقام الصفحات والروابط باستخدام Regex ---
        # حذف أرقام الصفحات مثل 31/37
        text = re.sub(r'\d+/\d+', '', text)
        
        # حذف أي رابط يبدأ بـ http (روابط منصة اعتماد)
        text = re.sub(r'https?://\S+', '', text)

        # 4. تنظيف الفراغات الناتجة عن الحذف
        text = clean_extra_whitespace(text)

        # إضافة النص النظيف فقط إذا لم يصبح فارغاً بعد الحذف
        if text.strip():
            cleaned_text.append(text)

    return "\n\n".join(cleaned_text)

# --- التشغيل ---
input_file = r"C:\Users\user\OneDrive - University of Prince Mugrin\سطح المكتب\Capstone_Project_SDAIA\src\data\processed\request1_raw.md"
final_content = auto_clean_with_unstructured(input_file)

with open("request1_final_clean3.md", "w", encoding="utf-8") as f:
    f.write(final_content)

print("✅ تم تنظيف الملف بنجاح وحذف أوصاف الصور والروابط وأرقام الصفحات!")

In [ ]:

def read_toc_only(file_path, limit_lines=145):
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = [next(f) for _ in range(limit_lines)]
    return "".join(lines)

# ثم نمرر هذا النص للـ Task مباشرة
toc_content = read_toc_only(r"C:\Users\user\OneDrive - University of Prince Mugrin\سطح المكتب\Capstone_Project_SDAIA\src\Agents\request1_final_clean3.md")

In [ ]:
print(toc_content)

In [ ]:
import json
import os

def read_toc_to_json(file_path, output_json_path, limit_lines=145):
    lines = []
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            # نقرأ الأسطر ونقوم بتنظيفها من الفراغات الزائدة
            for _ in range(limit_lines):
                line = next(f).strip()
                if line: # نتجاهل الأسطر الفارغة تماماً لتقليل حجم الملف
                    lines.append(line)
    except StopIteration:
        pass # انتهى الملف قبل الوصول للحد المحدد
    except Exception as e:
        print(f"حدث خطأ أثناء القراءة: {e}")
        return None

    # تشكيل هيكل البيانات للـ JSON
    toc_data = {
        "document_name": os.path.basename(file_path),
        "toc_lines": lines  # هنا نضع القائمة
    }

    # حفظ الملف بصيغة JSON
    with open(output_json_path, 'w', encoding='utf-8') as json_file:
        json.dump(toc_data, json_file, ensure_ascii=False, indent=4)
    
    print(f"✅ تم استخراج الفهرس وحفظه في: {output_json_path}")
    return toc_data

# --- طريقة الاستخدام ---
input_md = r"C:\Users\user\OneDrive - University of Prince Mugrin\سطح المكتب\Capstone_Project_SDAIA\src\Agents\request1_final_clean3.md"
output_json = "toc_extracted.json"

# تشغيل الدالة
toc_content = read_toc_to_json(input_md, output_json)